In [2]:
import requests
import os

# Renfe Horarios de alta velocidad, larga distancia y media distancia
url = 'https://ssl.renfe.com/gtransit/Fichero_AV_LD/google_transit.zip'

# Download zip file
req = requests.get(url)

# Get file name from url
filename = url.split('/')[-1]

savepath = "datasets/"

if not os.path.exists(savepath):
   os.makedirs(savepath)

# Writing the file to the local file system
with open(savepath+filename, 'wb') as output_file:
    output_file.write(req.content)

print('Downloading Completed')

In [3]:
import shutil

# Unzip zip
shutil.unpack_archive(savepath+filename, savepath)

In [4]:
import pandas as pd

agency = pd.read_csv(savepath+"/agency.txt", sep=",")

agency.head()

,agency_id,agency_name,agency_url,agency_timezone,agency_lang,agency_phone
0,1071,RENFE OPERADORA,http://www.renfe.com,Europe/Madrid,ES,902320320


In [5]:
calendar_dates = pd.read_csv(savepath+"/calendar_dates.txt", sep=",")

calendar_dates.head()

,service_id,date,exception_type
0,2022-11-302022-12-10001901,20221204,2
1,2022-12-112023-03-12001901,20221211,2
2,2022-12-112023-03-12001901,20221218,2
3,2022-12-112023-03-12001901,20221225,2
4,2022-12-112023-03-12001901,20230101,2


In [6]:
import datetime

# Parse date column
calendar_dates.date = calendar_dates.date.apply(lambda d : datetime.datetime.strptime(str(d), '%Y%m%d'))

In [7]:
calendar_dates.head()

,service_id,date,exception_type
0,2022-11-302022-12-10001901,2022-12-04,2
1,2022-12-112023-03-12001901,2022-12-11,2
2,2022-12-112023-03-12001901,2022-12-18,2
3,2022-12-112023-03-12001901,2022-12-25,2
4,2022-12-112023-03-12001901,2023-01-01,2


In [8]:
res = []

# Iterate directory
for filename in os.listdir(savepath):
    # check only text files
    if filename.endswith('.txt'):
        res.append(filename)
print(res)

['agency.txt', 'calendar_dates.txt', 'stop_times.txt', 'trips.txt', 'stops.txt', 'calendar.txt', 'routes.txt']


In [12]:
renfe_schedules = {}

for filename in os.listdir(savepath):
    if filename.endswith('.txt'):
        renfe_schedules[filename.split(".")[0]] = pd.read_csv(savepath+filename, delimiter=',')

In [13]:
renfe_schedules.keys()

dict_keys(['agency', 'calendar_dates', 'stop_times', 'trips', 'stops', 'calendar', 'routes'])